In [12]:
!pip install tensorflow matplotlib numpy opencv-python scikit-learn


In [13]:
from tensorflow.keras.datasets import cifar10
import numpy as np

(x_train, _), (x_test, _) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


In [14]:
import cv2

def rgb2lab_batch(images):
    lab_images = []
    for img in images:
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        lab_images.append(lab)
    return np.array(lab_images)

x_train_lab = rgb2lab_batch(x_train)
x_test_lab = rgb2lab_batch(x_test)

x_train_l = x_train_lab[:,:,:,0:1] / 100.0
x_train_ab = x_train_lab[:,:,:,1:] / 128.0

x_test_l = x_test_lab[:,:,:,0:1] / 100.0
x_test_ab = x_test_lab[:,:,:,1:] / 128.0


In [15]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def build_unet_model(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder
    c1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)

    c2 = Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)

    c3 = Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(256, 3, activation='relu', padding='same')(c3)

    # Decoder
    u1 = UpSampling2D()(c3)
    u1 = concatenate([u1, c2])
    c4 = Conv2D(128, 3, activation='relu', padding='same')(u1)

    u2 = UpSampling2D()(c4)
    u2 = concatenate([u2, c1])
    c5 = Conv2D(64, 3, activation='relu', padding='same')(u2)

    outputs = Conv2D(2, 1, activation='tanh')(c5)
    model = Model(inputs, outputs)
    return model

model = build_unet_model((32, 32, 1))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


In [16]:
history = model.fit(
    x_train_l, x_train_ab,
    validation_data=(x_test_l, x_test_ab),
    batch_size=128,
    epochs=25
)


Epoch 1/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 33s 68ms/step - accuracy: 0.5980 - loss: 0.0116 - val_accuracy: 0.6183 - val_loss: 0.0102
Epoch 2/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 54ms/step - accuracy: 0.6295 - loss: 0.0101 - val_accuracy: 0.6505 - val_loss: 0.0093
Epoch 3/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 57ms/step - accuracy: 0.6499 - loss: 0.0094 - val_accuracy: 0.6530 - val_loss: 0.0091
Epoch 4/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.6562 - loss: 0.0092 - val_accuracy: 0.6531 - val_loss: 0.0091
Epoch 5/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.6593 - loss: 0.0090 - val_accuracy: 0.6542 - val_loss: 0.0089
Epoch 6/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 55ms/step - accuracy: 0.6607 - loss: 0.0089 - val_accuracy: 0.6638 - val_loss: 0.0088
Epoch 7/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - accuracy: 0.6655 - loss: 0.0087 - val_accuracy: 0.6677 - val_loss: 0.0087
Epoch 8/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - accuracy: 0.6675 - loss: 0.0087 - 

In [18]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

pred_ab = model.predict(x_test_l)
pred_ab = np.clip(pred_ab, -1, 1)

# Rescale and convert back to RGB
def lab_to_rgb_batch(l, ab):
    l = l * 100
    ab = ab * 128
    lab = np.concatenate([l, ab], axis=-1)
    rgb_images = []
    for i in range(lab.shape[0]):
        rgb = cv2.cvtColor(lab[i].astype("float32"), cv2.COLOR_LAB2RGB)
        rgb_images.append(np.clip(rgb, 0, 1))
    return np.array(rgb_images)

x_test_rgb_true = lab_to_rgb_batch(x_test_l, x_test_ab)
x_test_rgb_pred = lab_to_rgb_batch(x_test_l, pred_ab)

# Flatten for metrics
y_true = (x_test_rgb_true * 255).astype(np.uint8).flatten()
y_pred = (x_test_rgb_pred * 255).astype(np.uint8).flatten()

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Confusion Matrix:
[[ 50158  14008   7243 ...      0      0      0]
 [ 16236  15914   8817 ...      0      0      0]
 [  3341   6482  12089 ...      0      0      0]
 ...
 [     0      0      0 ...    462    496    156]
 [     0      0      0 ...  29622  72724  10125]
 [     0      0      0 ...   8064  22005 135535]]
Precision: 0.09168662954730993
Recall: 0.08422992649412472


In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.07775546875


In [22]:
model.save("colorization_model.h5")


In [23]:
from google.colab import drive
drive.mount('/content/drive')
# Then manually move the file to your Drive


Mounted at /content/drive
